In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

In [2]:
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb

In [3]:
train=pd.read_csv('train.csv',encoding="utf-8")
test=pd.read_csv('test.csv',encoding="utf-8")
#sample_submission=pd.read_csv('sample_submission.csv',encoding="utf-8")   

In [4]:
train.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"


## Camera 총 이동 거리 측정

In [6]:
train_camera = train[train["event"]=="Camera"]
test_camera = test[test["event"]=="Camera"]

In [9]:
train_ca0 = train_camera[train_camera["player"] == 0]
train_ca1 = train_camera[train_camera["player"] == 1]
test_ca0 = test_camera[test_camera["player"] == 0]
test_ca1 = test_camera[test_camera["player"] == 1]


In [10]:
train_ca0.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"
5,0,1,0.14,0,T,Camera,"at (142.5078125, 24.98828125)"
6,0,1,0.14,0,T,Camera,"at (139.6171875, 27.8828125)"
7,0,1,0.14,0,T,Camera,"at (138.3359375, 29.1640625)"


In [11]:
train_ca0.shape

(15834817, 7)

In [12]:
player_0_camera = np.array([item.replace('at (', '').replace(')', '').split(',') for item in train_ca0['event_contents']]).astype(float)
player_1_camera = np.array([item.replace('at (', '').replace(')', '').split(',') for item in train_ca1['event_contents']]).astype(float)

In [13]:
player_0_camera

array([[145.25      ,  21.5078125 ],
       [142.99609375,  24.50390625],
       [142.5078125 ,  24.98828125],
       ...,
       [139.578125  ,  62.58203125],
       [122.42578125,  45.4296875 ],
       [122.42578125,  43.25390625]])

In [14]:
df_c0 = pd.DataFrame(player_0_camera,columns=['player_0_camera_x','player_0_camera_y'])

In [15]:
df_c1 = pd.DataFrame(player_1_camera,columns=['player_1_camera_x','player_1_camera_y'])

In [16]:
df_c1

,player_1_camera_x,player_1_camera_y
0,22.750000,147.007812
1,22.750000,145.906250
2,22.750000,143.246094
3,22.750000,140.992188
4,23.132812,140.992188
...,...,...
15794585,46.507812,113.257812
15794586,47.609375,113.257812
15794587,48.789062,113.257812
15794588,51.039062,115.511719


In [49]:
df_c0.to_csv("camera0.csv",encoding="utf-8",index=False)
df_c1.to_csv("camera1.csv",encoding="utf-8",index=False)

In [19]:
train_ca0.reset_index(inplace=True)
train_ca1.reset_index(inplace=True)

In [20]:
train_ca0["p0_cam_x"] = df_c0["player_0_camera_x"]
train_ca0["p0_cam_y"] = df_c0["player_0_camera_y"]

train_ca1["p1_cam_x"] = df_c1["player_1_camera_x"]
train_ca1["p1_cam_y"] = df_c1["player_1_camera_y"]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [21]:
train_ca0

,index,game_id,winner,time,player,species,event,event_contents,p0_cam_x,p0_cam_y
0,0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)",145.250000,21.507812
1,4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)",142.996094,24.503906
2,5,0,1,0.14,0,T,Camera,"at (142.5078125, 24.98828125)",142.507812,24.988281
3,6,0,1,0.14,0,T,Camera,"at (139.6171875, 27.8828125)",139.617188,27.882812
4,7,0,1,0.14,0,T,Camera,"at (138.3359375, 29.1640625)",138.335938,29.164062
...,...,...,...,...,...,...,...,...,...,...
15834812,67091763,38871,0,8.50,0,Z,Camera,"at (145.59375, 63.22265625)",145.593750,63.222656
15834813,67091764,38871,0,8.50,0,Z,Camera,"at (143.67578125, 63.22265625)",143.675781,63.222656
15834814,67091771,38871,0,8.51,0,Z,Camera,"at (139.578125, 62.58203125)",139.578125,62.582031
15834815,67091773,38871,0,8.52,0,Z,Camera,"at (122.42578125, 45.4296875)",122.425781,45.429688


In [22]:
train_ca0.groupby('game_id').diff()

,index,winner,time,player,p0_cam_x,p0_cam_y
0,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,0.0,0.14,0.0,-2.253906,2.996094
2,1.0,0.0,0.00,0.0,-0.488281,0.484375
3,1.0,0.0,0.00,0.0,-2.890625,2.894531
4,1.0,0.0,0.00,0.0,-1.281250,1.281250
...,...,...,...,...,...,...
15834812,3.0,0.0,0.00,0.0,-11.906250,13.566406
15834813,1.0,0.0,0.00,0.0,-1.917969,0.000000
15834814,7.0,0.0,0.01,0.0,-4.097656,-0.640625
15834815,2.0,0.0,0.01,0.0,-17.152344,-17.152344


In [25]:
train_ca0.head()

,index,game_id,winner,time,player,species,event,event_contents,p0_cam_x,p0_cam_y
0,0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)",145.250000,21.507812
1,4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)",142.996094,24.503906
2,5,0,1,0.14,0,T,Camera,"at (142.5078125, 24.98828125)",142.507812,24.988281
3,6,0,1,0.14,0,T,Camera,"at (139.6171875, 27.8828125)",139.617188,27.882812
4,7,0,1,0.14,0,T,Camera,"at (138.3359375, 29.1640625)",138.335938,29.164062


In [28]:
train_ca0[:1000].to_csv("test_look.csv",encoding="utf-8",index=False)

In [26]:
train_ca0['diff_p0_cam_x'] = train_ca0.groupby(['game_id'])['p0_cam_x'].diff().fillna(0)
train_ca0['diff_p0_cam_y'] = train_ca0.groupby(['game_id'])['p0_cam_y'].diff().fillna(0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
train_ca1['diff_p1_cam_x'] = train_ca1.groupby(['game_id'])['p1_cam_x'].diff().fillna(0)
train_ca1['diff_p1_cam_y'] = train_ca1.groupby(['game_id'])['p1_cam_y'].diff().fillna(0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
train_ca0['diff_p0_cam_x'] = abs(train_ca0['diff_p0_cam_x'])
train_ca0['diff_p0_cam_y'] = abs(train_ca0['diff_p0_cam_y'])
train_ca1['diff_p1_cam_x'] = abs(train_ca1['diff_p1_cam_x'])
train_ca1['diff_p1_cam_y'] = abs(train_ca1['diff_p1_cam_y'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [38]:
t0 = train_ca0.groupby('game_id').sum()
t1 = train_ca1.groupby('game_id').sum()

In [58]:
t0

,index,winner,time,player,p0_cam_x,p0_cam_y,diff_p0_cam_x,diff_p0_cam_y
game_id,,,,,,,,
0,245224,444,1681.39,0,58596.792969,20456.996094,545.644531,975.710938
1,1558366,627,3463.11,0,79442.585938,89684.417969,1852.785156,3296.531250
2,1934754,0,2136.92,0,52514.093750,21472.316406,2448.265625,4145.347656
3,5374592,0,3966.58,0,72339.234375,41874.742188,2705.027344,3627.335938
4,4169723,0,1302.00,0,30017.691406,43512.527344,1954.074219,1618.089844
...,...,...,...,...,...,...,...,...
38867,42866231878,639,2494.83,0,24171.058594,76397.683594,496.019531,1567.285156
38868,37433296372,0,2610.25,0,28479.289062,24292.746094,645.945312,903.726562
38869,52663013159,0,3578.62,0,104385.707031,31346.937500,1980.082031,2345.640625


In [57]:
t1

,index,winner,time,player,p1_cam_x,p1_cam_y,diff_p0_cam_x,diff_p0_cam_y,diff_p1_cam_x,diff_p1_cam_y
game_id,,,,,,,,,,
0,239636,425,1640.86,425,11550.996094,57644.296875,5.593750,-6.218750,314.554688,653.351562
1,2038161,858,4375.75,858,49854.496094,44596.464844,8.019531,31.378906,3544.128906,4668.597656
2,3426194,0,3926.06,725,36986.500000,71121.937500,35.890625,-18.417969,3679.734375,3208.253906
3,2934668,0,2196.23,388,20707.386719,40224.449219,88.402344,-88.488281,2354.542969,3795.207031
4,2571231,0,588.68,272,38596.218750,11192.035156,-32.109375,15.410156,3640.109375,4107.105469
...,...,...,...,...,...,...,...,...,...,...
38867,21735061697,324,1707.70,324,38247.339844,15841.460938,-4.367188,25.613281,415.054688,767.480469
38868,42464694926,0,3184.38,633,77992.312500,88369.964844,-112.000000,-114.902344,598.773438,999.003906
38869,22138605175,0,1587.68,330,19500.628906,27639.718750,131.250000,-37.789062,3434.554688,3847.992188


In [32]:
test_ca0.reset_index(inplace=True)
test_ca1.reset_index(inplace=True)

In [ ]:
player_0_camera = np.array([item.replace('at (', '').replace(')', '').split(',') for item in train_ca0['event_contents']]).astype(float)
player_1_camera = np.array([item.replace('at (', '').replace(')', '').split(',') for item in train_ca1['event_contents']]).astype(float)

In [121]:
test_ca0["p0_cam_x"] = df_diff_c0["p0_cam_x"]
test_ca0["p0_cam_y"] = df_diff_c0["p0_cam_y"]
test_ca1["p1_cam_x"] = df_diff_c1["p1_cam_x"]
test_ca1["p1_cam_y"] = df_diff_c1["p1_cam_y"]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [122]:
test_ca0

,index,game_id,time,player,species,event,event_contents,p0_cam_x,p0_cam_y
0,0,38872,0.00,0,P,Camera,"at (22.25, 81.5078125)",NaN,NaN
1,46,38872,0.16,0,P,Camera,"at (24.9453125, 78.8125)",2.695312,2.695312
2,48,38872,0.16,0,P,Camera,"at (27.80859375, 75.94921875)",2.863281,2.863281
3,49,38872,0.16,0,P,Camera,"at (30.50390625, 75.94921875)",2.695312,0.000000
4,51,38872,0.16,0,P,Camera,"at (33.2578125, 75.94921875)",2.753906,0.000000
...,...,...,...,...,...,...,...,...,...
6776620,28714749,55658,4.25,0,Z,Camera,"at (88.9765625, 67.609375)",1.160156,0.589844
6776621,28714750,55658,4.25,0,Z,Camera,"at (88.39453125, 67.609375)",0.582031,0.000000
6776622,28714818,55658,4.42,0,Z,Camera,"at (133.06640625, 30.51953125)",44.671875,37.089844
6776623,28714833,55658,4.49,0,Z,Camera,"at (126.10546875, 42.29296875)",6.960938,11.773438


In [56]:
train_ca0

,index,game_id,winner,time,player,species,event,event_contents,p0_cam_x,p0_cam_y,diff_p0_cam_x,diff_p0_cam_y
0,0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)",145.250000,21.507812,0.000000,0.000000
1,4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)",142.996094,24.503906,2.253906,2.996094
2,5,0,1,0.14,0,T,Camera,"at (142.5078125, 24.98828125)",142.507812,24.988281,0.488281,0.484375
3,6,0,1,0.14,0,T,Camera,"at (139.6171875, 27.8828125)",139.617188,27.882812,2.890625,2.894531
4,7,0,1,0.14,0,T,Camera,"at (138.3359375, 29.1640625)",138.335938,29.164062,1.281250,1.281250
...,...,...,...,...,...,...,...,...,...,...,...,...
15834812,67091763,38871,0,8.50,0,Z,Camera,"at (145.59375, 63.22265625)",145.593750,63.222656,11.906250,13.566406
15834813,67091764,38871,0,8.50,0,Z,Camera,"at (143.67578125, 63.22265625)",143.675781,63.222656,1.917969,0.000000
15834814,67091771,38871,0,8.51,0,Z,Camera,"at (139.578125, 62.58203125)",139.578125,62.582031,4.097656,0.640625
15834815,67091773,38871,0,8.52,0,Z,Camera,"at (122.42578125, 45.4296875)",122.425781,45.429688,17.152344,17.152344


In [98]:
train_ca1

,index,game_id,winner,time,player,species,event,event_contents,p1_cam_y,p1_cam_x
0,1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)",NaN,NaN
1,30,0,1,0.34,1,T,Camera,"at (22.75, 145.90625)",1.101562,0.000000
2,31,0,1,0.34,1,T,Camera,"at (22.75, 143.24609375)",2.660156,0.000000
3,33,0,1,0.34,1,T,Camera,"at (22.75, 140.9921875)",2.253906,0.000000
4,34,0,1,0.35,1,T,Camera,"at (23.1328125, 140.9921875)",0.000000,0.382812
...,...,...,...,...,...,...,...,...,...,...
15794585,67091761,38871,0,8.50,1,T,Camera,"at (46.5078125, 113.2578125)",0.000000,1.152344
15794586,67091762,38871,0,8.50,1,T,Camera,"at (47.609375, 113.2578125)",0.000000,1.101562
15794587,67091767,38871,0,8.51,1,T,Camera,"at (48.7890625, 113.2578125)",0.000000,1.179688
15794588,67091769,38871,0,8.51,1,T,Camera,"at (51.0390625, 115.51171875)",2.253906,2.250000


In [44]:
data = train["game_id"].unique()

In [45]:
df = pd.DataFrame(data,columns = ['game_id'])

In [47]:
tdata = test["game_id"].unique()

In [49]:
tdata.shape

(16787,)

In [87]:
df_t = pd.DataFrame(tdata,columns = ['game_id'])

In [64]:
tc00 = t0[["diff_p0_cam_x","diff_p0_cam_y"]]
tc11 = t1[["diff_p1_cam_x","diff_p1_cam_y"]]

In [85]:
te00 = te0[["diff_p0_cam_x","diff_p0_cam_y"]]
te11 = te1[["diff_p1_cam_x","diff_p1_cam_y"]]

In [86]:
te00

,diff_p0_cam_x,diff_p0_cam_y
game_id,,
38872,881.546875,878.964844
38873,1668.277344,1851.167969
38874,2420.000000,2234.027344
38875,5158.632812,5370.031250
38876,609.292969,296.570312
...,...,...
55654,2987.273438,2822.640625
55655,1709.972656,2604.707031
55656,2184.058594,1278.421875


In [68]:
df = pd.merge(df, tc00, on='game_id', how='left')

In [70]:
df = pd.merge(df, tc11, on='game_id', how='left')

In [88]:
df_t = pd.merge(df_t, te00, on='game_id', how='left')

In [89]:
df_t = pd.merge(df_t, te11, on='game_id', how='left')

In [90]:
df_t

,game_id,diff_p0_cam_x,diff_p0_cam_y,diff_p1_cam_x,diff_p1_cam_y
0,38872,881.546875,878.964844,2246.074219,2550.453125
1,38873,1668.277344,1851.167969,1288.992188,1433.558594
2,38874,2420.000000,2234.027344,499.804688,659.636719
3,38875,5158.632812,5370.031250,2246.640625,2854.953125
4,38876,609.292969,296.570312,43.082031,274.597656
...,...,...,...,...,...
16782,55654,2987.273438,2822.640625,3202.710938,2522.371094
16783,55655,1709.972656,2604.707031,1103.027344,1067.875000
16784,55656,2184.058594,1278.421875,2431.421875,3190.503906
16785,55657,0.000000,31.593750,444.339844,633.484375


In [91]:
df_t["diff_p0_cam"] = df_t["diff_p0_cam_x"] + df_t["diff_p0_cam_y"]
df_t["diff_p1_cam"] = df_t["diff_p1_cam_x"] + df_t["diff_p1_cam_y"]

In [92]:
df_t.to_csv("cam_data_test.csv",index=False,encoding="utf-8")

In [72]:
df["diff_p0_cam"] = df["diff_p0_cam_x"] + df["diff_p0_cam_y"]
df["diff_p1_cam"] = df["diff_p1_cam_x"] + df["diff_p1_cam_y"]

In [73]:
df

,game_id,diff_p0_cam_x,diff_p0_cam_y,diff_p1_cam_x,diff_p1_cam_y,diff_p0_cam,diff_p1_cam
0,0,545.644531,975.710938,314.554688,653.351562,1521.355469,967.906250
1,1,1852.785156,3296.531250,3544.128906,4668.597656,5149.316406,8212.726562
2,2,2448.265625,4145.347656,3679.734375,3208.253906,6593.613281,6887.988281
3,3,2705.027344,3627.335938,2354.542969,3795.207031,6332.363281,6149.750000
4,4,1954.074219,1618.089844,3640.109375,4107.105469,3572.164062,7747.214844
...,...,...,...,...,...,...,...
38867,38867,496.019531,1567.285156,415.054688,767.480469,2063.304688,1182.535156
38868,38868,645.945312,903.726562,598.773438,999.003906,1549.671875,1597.777344
38869,38869,1980.082031,2345.640625,3434.554688,3847.992188,4325.722656,7282.546875
38870,38870,86.828125,1353.171875,1378.308594,1888.726562,1440.000000,3267.035156


In [74]:
df.to_csv("cam_data.csv",index=False,encoding="utf-8")

In [140]:
df = df.fillna(0)

In [103]:
df.drop(["p1_cam_x","p1_cam_y"],1,inplace=True)

In [141]:
df.to_csv("test_cam_var.csv",encoding="utf-8",index=False)

In [112]:
df.to_csv("train_cam_var.csv",encoding="utf-8",index=False)

In [75]:
player_te0_camera = np.array([item.replace('at (', '').replace(')', '').split(',') for item in test_ca0['event_contents']]).astype(float)
player_te1_camera = np.array([item.replace('at (', '').replace(')', '').split(',') for item in test_ca1['event_contents']]).astype(float)

In [115]:
player_te0_camera

array([[ 22.25      ,  81.5078125 ],
       [ 24.9453125 ,  78.8125    ],
       [ 27.80859375,  75.94921875],
       ...,
       [133.06640625,  30.51953125],
       [126.10546875,  42.29296875],
       [ 70.9921875 , 117.65234375]])

In [76]:
df_test_c0 = pd.DataFrame(player_te0_camera,columns=['p0_cam_x','p0_cam_y'])
df_test_c1 = pd.DataFrame(player_te1_camera,columns=['p1_cam_x','p1_cam_y'])

In [77]:
df_test_c0

,p0_cam_x,p0_cam_y
0,22.250000,81.507812
1,24.945312,78.812500
2,27.808594,75.949219
3,30.503906,75.949219
4,33.257812,75.949219
...,...,...
6776620,88.976562,67.609375
6776621,88.394531,67.609375
6776622,133.066406,30.519531
6776623,126.105469,42.292969


In [78]:
test_ca0.reset_index(inplace=True)
test_ca1.reset_index(inplace=True)

In [79]:
test_ca0["p0_cam_x"] = df_test_c0["p0_cam_x"]
test_ca0["p0_cam_y"] = df_test_c0["p0_cam_y"]

test_ca1["p1_cam_x"] = df_test_c1["p1_cam_x"]
test_ca1["p1_cam_y"] = df_test_c1["p1_cam_y"]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [80]:
test_ca0

,level_0,index,game_id,time,player,species,event,event_contents,p0_cam_x,p0_cam_y
0,0,0,38872,0.00,0,P,Camera,"at (22.25, 81.5078125)",22.250000,81.507812
1,1,46,38872,0.16,0,P,Camera,"at (24.9453125, 78.8125)",24.945312,78.812500
2,2,48,38872,0.16,0,P,Camera,"at (27.80859375, 75.94921875)",27.808594,75.949219
3,3,49,38872,0.16,0,P,Camera,"at (30.50390625, 75.94921875)",30.503906,75.949219
4,4,51,38872,0.16,0,P,Camera,"at (33.2578125, 75.94921875)",33.257812,75.949219
...,...,...,...,...,...,...,...,...,...,...
6776620,6776620,28714749,55658,4.25,0,Z,Camera,"at (88.9765625, 67.609375)",88.976562,67.609375
6776621,6776621,28714750,55658,4.25,0,Z,Camera,"at (88.39453125, 67.609375)",88.394531,67.609375
6776622,6776622,28714818,55658,4.42,0,Z,Camera,"at (133.06640625, 30.51953125)",133.066406,30.519531
6776623,6776623,28714833,55658,4.49,0,Z,Camera,"at (126.10546875, 42.29296875)",126.105469,42.292969


In [81]:
test_ca0['diff_p0_cam_x'] = test_ca0.groupby(['game_id'])['p0_cam_x'].diff().fillna(0)
test_ca0['diff_p0_cam_y'] = test_ca0.groupby(['game_id'])['p0_cam_y'].diff().fillna(0)

test_ca1['diff_p1_cam_x'] = test_ca1.groupby(['game_id'])['p1_cam_x'].diff().fillna(0)
test_ca1['diff_p1_cam_y'] = test_ca1.groupby(['game_id'])['p1_cam_y'].diff().fillna(0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [82]:
test_ca0

,level_0,index,game_id,time,player,species,event,event_contents,p0_cam_x,p0_cam_y,diff_p0_cam_x,diff_p0_cam_y
0,0,0,38872,0.00,0,P,Camera,"at (22.25, 81.5078125)",22.250000,81.507812,0.000000,0.000000
1,1,46,38872,0.16,0,P,Camera,"at (24.9453125, 78.8125)",24.945312,78.812500,2.695312,-2.695312
2,2,48,38872,0.16,0,P,Camera,"at (27.80859375, 75.94921875)",27.808594,75.949219,2.863281,-2.863281
3,3,49,38872,0.16,0,P,Camera,"at (30.50390625, 75.94921875)",30.503906,75.949219,2.695312,0.000000
4,4,51,38872,0.16,0,P,Camera,"at (33.2578125, 75.94921875)",33.257812,75.949219,2.753906,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
6776620,6776620,28714749,55658,4.25,0,Z,Camera,"at (88.9765625, 67.609375)",88.976562,67.609375,-1.160156,0.589844
6776621,6776621,28714750,55658,4.25,0,Z,Camera,"at (88.39453125, 67.609375)",88.394531,67.609375,-0.582031,0.000000
6776622,6776622,28714818,55658,4.42,0,Z,Camera,"at (133.06640625, 30.51953125)",133.066406,30.519531,44.671875,-37.089844
6776623,6776623,28714833,55658,4.49,0,Z,Camera,"at (126.10546875, 42.29296875)",126.105469,42.292969,-6.960938,11.773438


In [83]:
test_ca0['diff_p0_cam_x'] = abs(test_ca0['diff_p0_cam_x'])
test_ca0['diff_p0_cam_y'] = abs(test_ca0['diff_p0_cam_y'])
test_ca1['diff_p1_cam_x'] = abs(test_ca1['diff_p1_cam_x'])
test_ca1['diff_p1_cam_y'] = abs(test_ca1['diff_p1_cam_y'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [84]:
te0 = test_ca0.groupby('game_id').sum()
te1 = test_ca1.groupby('game_id').sum()